# Generacion de chunks

In [2]:
import re
import json
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_and_split_docs(filepath="/Users/raldo/Documents/LLMS/Taller1/info/base_conocimiento.txt"):
    """
    Carga el archivo de base de conocimiento y lo divide en documentos individuales
    basados en el delimitador '==='.
    """
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            content = f.read()
        # El delimitador es la línea que empieza con '=== '
        # Usamos una expresión regular para dividir manteniendo el delimitador para metadatos
        docs_raw = re.split(r'(?m)^=== .* ===$', content)
        # El primer elemento suele estar vacío, lo filtramos
        docs_raw = [doc for doc in docs_raw if doc.strip()]
        
        # Necesitamos obtener los nombres de archivo que actúan como delimitadores
        delimiters = re.findall(r'(?m)^=== (.*) ===$', content)

        # Reconstruimos los documentos con su delimitador para el parsing
        processed_docs = []
        for i, doc_content in enumerate(docs_raw):
            if i < len(delimiters):
                full_doc = f"=== {delimiters[i]} ===\n{doc_content.strip()}"
                processed_docs.append(full_doc)

        print(f"Archivo principal cargado. Se encontraron {len(processed_docs)} documentos individuales.")
        return processed_docs
    except FileNotFoundError:
        print(f"Error: El archivo {filepath} no fue encontrado.")
        return []

def parse_document(doc_raw):
    """
    Extrae los metadatos y el contenido principal de un documento crudo.
    """
    metadata = {}
    content = ""
    lines = doc_raw.split('\n')

    # Extraer el nombre del archivo del delimitador
    filename_match = re.search(r'=== (.*) ===', lines[0])
    if filename_match:
        metadata['source_file'] = filename_match.group(1).strip()

    # Extraer metadatos y contenido principal
    in_content_section = False
    for line in lines[1:]: # Omitir la primera línea (delimitador)
        if line.strip() == 'Contenido Principal':
            in_content_section = True
            continue
        
        if line.startswith(('Estructura de Headings', 'Información de Contacto', 'Redes Sociales')):
            in_content_section = False # Se terminó la sección de contenido principal

        if in_content_section:
            content += line + '\n'
        else:
            # Parsear metadatos clave-valor
            if ':' in line:
                key, *value = line.split(':', 1)
                key = key.strip().lower().replace(' ', '_')
                # Ignorar campos no deseados en los metadatos principales
                if key in ['url', 'title', 'description', 'keywords', 'scraped_date', 'source']:
                    metadata[key] = ''.join(value).strip()

    return content.strip(), metadata

def main():
    """
    Función principal que orquesta el proceso de carga, parsing y chunking.
    """
    # 1. Cargar y dividir el archivo en documentos
    documents_raw = load_and_split_docs()
    
    if not documents_raw:
        return

    all_texts = []
    all_metadatas = []

    # 2. Parsear cada documento para extraer contenido y metadatos
    for doc_raw in documents_raw:
        content, metadata = parse_document(doc_raw)
        if content and metadata.get('title'): # Solo procesar si tiene contenido y un título
            all_texts.append(content)
            all_metadatas.append(metadata)

    print(f"Se procesaron y parsearon {len(all_texts)} documentos con contenido útil.")

    # 3. Inicializar el Text Splitter con nuestra estrategia
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        separators=["\n\n", "\n", " ", ""] # Prioriza separación por párrafos
    )

    # 4. Aplicar el chunking a todos los textos
    # Langchain puede manejar la creación de documentos con sus metadatos asociados
    # de forma muy eficiente
    chunks = text_splitter.create_documents(all_texts, metadatas=all_metadatas)

    print(f"\n¡Chunking completado! Se generaron {len(chunks)} fragmentos en total.")

    # 5. Preparar los chunks para guardarlos en un archivo JSON
    output_chunks = []
    for i, chunk in enumerate(chunks):
        output_chunks.append({
            'chunk_id': i,
            'content': chunk.page_content,
            'metadata': chunk.metadata
        })

    # 6. Guardar los chunks en un archivo JSON
    output_filepath = "chunks.json"
    with open(output_filepath, 'w', encoding='utf-8') as f:
        json.dump(output_chunks, f, indent=2, ensure_ascii=False)

    print(f"Los chunks se han guardado exitosamente en el archivo: '{output_filepath}'")
    
    # 7. Mostrar un ejemplo de los primeros chunks para verificación
    print("\n--- Ejemplo de los primeros 2 Chunks ---")
    print(json.dumps(output_chunks[:2], indent=2, ensure_ascii=False))
    print("------------------------------------------")


if __name__ == "__main__":
    main()

Archivo principal cargado. Se encontraron 54 documentos individuales.
Se procesaron y parsearon 26 documentos con contenido útil.

¡Chunking completado! Se generaron 130 fragmentos en total.
Los chunks se han guardado exitosamente en el archivo: 'chunks.json'

--- Ejemplo de los primeros 2 Chunks ---
[
  {
    "chunk_id": 0,
    "content": "Marcas\nColombina y Tajín® presentan: nuevo Remix Bon Bon Bum x Tajín, la fusión que Internet hizo realidad\njulio 18 2025\nCali, 18 de julio de 2025.\nLas icónicas marcas Bon Bon Bum y Tajín se unen para lanzar un producto que revoluciona el sabor: el Nuevo Remix Bon Bon Bum x Tajín, una combinación que une los sabores del Bon Bon Bum con el toque único de Tajín. Esta alianza representa la celebración de la cultura, la creatividad y la autenticidad latinoamericana.\nInicialmente, estará disponible en Colombia, Estados Unidos, Guatemala, El Salvador y Puerto Rico, con proyecciones de ventas que superan los 5 millones de dólares en su primer año, se 

# Q&A

In [ ]:
import json
from langchain_community.llms import Ollama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

def load_knowledge_base(filepath="chunks.json"):
    """
    Carga los chunks desde el archivo JSON y los concatena en un solo
    string de texto que servirá como contexto completo.
    """
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # Unimos todo el contenido de los chunks en un solo texto gigante.
        # Agregamos metadatos clave para dar más contexto a cada chunk.
        full_context = ""
        for item in data:
            # Incluimos el título y la fuente de cada documento antes de su contenido
            # para ayudar al LLM a contextualizar la información.
            title = item['metadata'].get('title', 'Fuente desconocida')
            source_file = item['metadata'].get('source_file', 'Documento sin nombre')
            
            full_context += f"--- Inicio del Documento: {title} (Archivo: {source_file}) ---\n"
            full_context += item['content']
            full_context += f"\n--- Fin del Documento: {title} ---\n\n"
            
        print(f"✅ Base de conocimiento cargada y consolidada. Total de caracteres: {len(full_context)}")
        return full_context
    except FileNotFoundError:
        print(f"❌ Error: El archivo '{filepath}' no fue encontrado.")
        return None
    except Exception as e:
        print(f"❌ Ocurrió un error al cargar la base de conocimiento: {e}")
        return None

def create_simple_qa_chain(knowledge_context, llm_model="gpt-oss"):
    """
    Crea una cadena de Q&A simple que inyecta todo el conocimiento
    en el prompt, sin usar un retriever.
    """
    print(f"\n🔄 Configurando la cadena Q&A con el modelo de Ollama: '{llm_model}'...")

    # 1. Inicializar el LLM a través de Ollama
    llm = Ollama(model=llm_model)

    # 2. Crear el prompt template
    # Este prompt ahora tiene un placeholder para el contexto completo y la pregunta.
    template = """
    Eres un asistente experto de la empresa Colombina. Responde la pregunta del usuario basándote estricta y únicamente en la siguiente base de conocimiento.
    Si la respuesta no se encuentra en la base de conocimiento, responde exactamente: "La información solicitada no se encuentra en mi base de conocimiento."
    No intentes inventar una respuesta. Sé conciso y directo.

    --- INICIO DE LA BASE DE CONOCIMIENTO ---
    {context}
    --- FIN DE LA BASE DE CONOCIMIENTO ---

    Pregunta del usuario:
    {question}

    Respuesta:
    """
    prompt = PromptTemplate.from_template(template)

    # 3. Construir la cadena
    # Es más simple: solo formatea el prompt con el contexto y la pregunta,
    # y luego pasa el resultado al LLM.
    simple_qa_chain = (
        prompt
        | llm
        | StrOutputParser()
    )
    
    print("✅ Cadena de Q&A simple configurada exitosamente.")
    return simple_qa_chain

def main():
    # Cargar toda la base de conocimiento en una sola variable
    knowledge_context = load_knowledge_base()
    if knowledge_context is None:
        return

    # Crear la cadena de Q&A
    qa_chain = create_simple_qa_chain(knowledge_context, llm_model="gpt-oss:20b")

    # Listado de 20 preguntas
    questions = [
        # ... (Las mismas 20 preguntas de la respuesta anterior) ...
        "¿En qué año se creó el Bon Bon Bum?",
        "¿Cómo se llama el programa de Colombina para acompañar a sus proveedores y emprendedores?",
        "¿Qué porcentaje de la energía eléctrica que utiliza Colombina en sus operaciones en Colombia proviene de fuentes renovables?",
        "¿Cuál es la certificación que han recibido las 5 fábricas de Colombia en relación con la gestión de residuos?",
        "¿Quién fue el fundador de Colombina?",
        "Describe la colaboración entre Bon Bon Bum y Tajín. ¿Qué producto lanzaron y cuáles eran las proyecciones de ventas?",
        "¿Cuáles son los principales logros de Colombina en materia de sostenibilidad relacionados con la energía y el agua?",
        "Según la política de protección de datos, ¿cuál es el procedimiento que debe seguir una persona si desea actualizar o rectificar su información personal?",
        "Resume la historia de la creación de la chupeta Bon Bon Bum. ¿Quién la creó y cuál fue su innovación principal?",
        "¿Qué es Colombina Energía S.A.S. E.S.P. y cuál es su función principal?",
        "Compara las alianzas de Colombina con Ramo y Postobón. ¿Qué productos icónicos se crearon en cada colaboración?",
        "¿Qué relación existe entre la certificación 'Sello Oro Equipares' y los valores corporativos de Colombina?",
        "Lista tres plantas de producción de Colombina, su ubicación y qué tipo de productos se fabrican en cada una.",
        "¿En qué países fuera de Colombia tiene Colombina plantas de producción, según la información proporcionada?",
        "Si soy un proveedor y no he logrado obtener mi certificado de retención a través del portal, ¿a qué correo electrónico debo escribir?",
        "¿Cuál es la política de Colombina respecto al uso de huevos libres de jaula y cuál es la meta para 2025?",
        "De acuerdo con la política de tratamiento de datos, ¿qué ocurre con la información de un candidato que no es seleccionado para un puesto de trabajo?",
        "¿Cuál fue la calificación que recibió Colombina al obtener la certificación Basura Cero ORO para sus plantas de helados?",
        "Menciona dos deportistas que han sido embajadores o imagen de campañas relacionadas con Bon Bon Bum.",
        "¿Cuál es el salario anual del presidente de Colombina?"
    ]

    print("\n--- 🚀 INICIANDO EVALUACIÓN DEL SISTEMA Q&A (MÉTODO IN-CONTEXT) 🚀 ---\n")
    
    for i, question in enumerate(questions, 1):
        print(f"--- Pregunta {i}/{len(questions)} ---")
        print(f"❓: {question}")
        
        # Invocar la cadena. Pasamos el contexto completo y la pregunta.
        # El LLM tardará más porque tiene que "leer" todo el contexto cada vez.
        answer = qa_chain.invoke({"context": knowledge_context, "question": question})
        
        print(f"🤖: {answer}")
        print("-" * (len(str(i)) + len(str(len(questions))) + 16))
        print("\n")


if __name__ == "__main__":
    main()

✅ Base de conocimiento cargada y consolidada. Total de caracteres: 130954

🔄 Configurando la cadena Q&A con el modelo de Ollama: 'gpt-oss:20b'...
✅ Cadena de Q&A simple configurada exitosamente.

--- 🚀 INICIANDO EVALUACIÓN DEL SISTEMA Q&A (MÉTODO IN-CONTEXT) 🚀 ---

--- Pregunta 1/20 ---
❓: ¿En qué año se creó el Bon Bon Bum?
🤖: No dispongo de información sobre el año en que se creó el Bon Bon Bum.
-------------------


--- Pregunta 2/20 ---
❓: ¿Cómo se llama el programa de Colombina para acompañar a sus proveedores y emprendedores?
🤖: **Programa Big Brother.**
-------------------


--- Pregunta 3/20 ---
❓: ¿Qué porcentaje de la energía eléctrica que utiliza Colombina en sus operaciones en Colombia proviene de fuentes renovables?
🤖: El 100 % de la energía eléctrica que emplea Colombina en sus operaciones dentro de Colombia proviene de fuentes renovables.
-------------------


--- Pregunta 4/20 ---
❓: ¿Cuál es la certificación que han recibido las 5 fábricas de Colombia en relación con la gestión de residuos?
🤖: 
-------------------


--- Pregunta 5/20 ---
❓: ¿Quién fue el fundador de Colombina?
🤖: El fundador de Colombina fue **Carlos Fernando de la Torre**.
-------------------


--- Pregunta 6/20 ---
❓: Describe la colaboración entre Bon Bon Bum y Tajín. ¿Qué producto lanzaron y cuáles eran las proyecciones de ventas?
🤖: 
-------------------


--- Pregunta 7/20 ---
❓: ¿Cuáles son los principales logros de Colombina en materia de sostenibilidad relacionados con la energía y el agua?
🤖: **Logros de Colombina en sostenibilidad energética**

| Área | Acción | Impacto concreto |
|------|--------|-----------------|
| **Energía renovable** | • Instalación de **más de 36 000 paneles solares** en la Granja Solar y en las cinco plantas de producción de Colombia.  <br>• La granja solar convierte la energía solar en electricidad 100 % renovable, que alimenta la producción de alimentos. | • **Reducción de huella de carbono del 45 %** desde 2015 (por tonelada producida). |
| **Energía eléctrica** | • Todas las fábricas de Colombia están certificadas con el **Sello Basura Cero** por la organización Basura Cero Colombia, lo que garantiza una gestión energética eficiente y la reducción de residuos que impactan en el consumo energético. | • Optimización del consumo eléctrico y mayor eficiencia en los procesos de producción. |

**Logros de Colombina en sostenibilidad hídrica**

| Área | Acción | Impacto concreto |
|------|--------|-----------------|
| **Consumo de agua** | • Implementación de medidas de eficiencia hídrica en todas las plantas. | • **Ahorro del 27 %** del consumo de agua desde 2015 (por tonelada producida). |
| **Tratamiento y reutilización de aguas** | • Instalación de **plantas de tratamiento de aguas residuales** que purifican el agua usada en la producción y la devuelven limpia. | • Agua tratada “tan limpia como la recibimos”, contribuyendo a la sostenibilidad del recurso hídrico. |

En resumen, Colombina ha logrado:

* **Energía**: generación de electricidad 100 % renovable mediante una extensa red solar, reducción del 45 % en huella de carbono y certificación Basura Cero en todas sus plantas.  
* **Agua**: reducción del 27 % en consumo hídrico y tratamiento eficaz de aguas residuales que devuelve el agua a la producción de forma limpia.
-------------------


--- Pregunta 8/20 ---
❓: Según la política de protección de datos, ¿cuál es el procedimiento que debe seguir una persona si desea actualizar o rectificar su información personal?
🤖: **Procedimiento para actualizar o rectificar la información personal**

1. **Identificar la información**  
   - Señala con claridad cuál(es) dato(s) personal(es) deseas corregir o actualizar.

2. **Iniciar la solicitud**  
   - Presenta una petición formal de rectificación o actualización a través del canal designado por la entidad (por ejemplo, un formulario en línea, correo electrónico o medio establecido por la organización).

3. **Verificar la información proporcionada**  
   - La entidad revisará la solicitud y verificará la exactitud de los datos nuevos o corregidos.

4. **Respuesta y acción**  
   - Si la entidad acepta la solicitud, procederá a actualizar o rectificar la información en los sistemas y bases de datos correspondientes.
   - Si no recibe respuesta dentro de un plazo razonable, la persona puede dirigirse a las autoridades competentes (por ejemplo, el organismo regulador de protección de datos de su país) para solicitar una intervención.

En resumen, la persona debe: **identificar la información a rectificar, enviar la solicitud de actualización a través del canal designado, esperar la verificación y corrección por parte de la entidad, y en caso de falta de respuesta, recurrir a las autoridades competentes**.
-------------------


--- Pregunta 9/20 ---
❓: Resume la historia de la creación de la chupeta Bon Bon Bum. ¿Quién la creó y cuál fue su innovación principal?
🤖: Lo siento, pero la información sobre la creación de la chupeta **Bon Bon Bum** no está disponible en la base de conocimiento.
-------------------


--- Pregunta 10/20 ---
❓: ¿Qué es Colombina Energía S.A.S. E.S.P. y cuál es su función principal?
🤖: 
--------------------


--- Pregunta 11/20 ---
❓: Compara las alianzas de Colombina con Ramo y Postobón. ¿Qué productos icónicos se crearon en cada colaboración?
🤖: No se encuentran referencias en la base de conocimiento sobre alianzas de Colombina con la marca **Ramo** ni con **Postobón**. En consecuencia, no dispongo de información sobre productos icónicos que se hayan creado en dichas colaboraciones.
--------------------


--- Pregunta 12/20 ---
❓: ¿Qué relación existe entre la certificación 'Sello Oro Equipares' y los valores corporativos de Colombina?
🤖: 
--------------------


--- Pregunta 13/20 ---
❓: Lista tres plantas de producción de Colombina, su ubicación y qué tipo de productos se fabrican en cada una.
🤖: **Tres de las plantas de producción de la comunidad de La Paila (Valle del Cauca, Colombia)**  

| # | Nombre de la planta | Ubicación | Producto principal |
|---|---------------------|-----------|--------------------|
| 1 | **Planta de Confitería** | La Paila, Valle del Cauca | Chocolates, caramelos, chips de chocolate y una variedad de galletas (de chocolate, maíz, queso y avena) |
| 2 | **Planta de Galletas** | La Paila, Valle del Cauca | Galletas de chocolate, maíz, queso y avena (producción especializada en diferentes tipos de galleta) |
| 3 | **Planta de Galletas (Granja Solar)** | La Paila, Valle del Cauca | Galletas de chocolate, maíz, queso y avena (generada dentro del mismo complejo agro‑solar, con énfasis en la producción de galletas sostenibles) |

> *Nota:* La comunidad de La Paila combina la producción tradicional de confitería con la fabricación de galletas, apoyada por la Granja Solar, que integra agricultura y energía renovable. Las tres plantas mencionadas son unidades de producción distintas, aunque dos comparten la misma localidad (La Paila).
--------------------


--- Pregunta 14/20 ---
❓: ¿En qué países fuera de Colombia tiene Colombina plantas de producción, según la información proporcionada?
🤖: Según la información incluida en la base de conocimiento, Colombina no dispone de plantas de producción fuera de Colombia.  Todas las fábricas mencionadas están ubicadas en territorio colombiano.
--------------------


--- Pregunta 15/20 ---
❓: Si soy un proveedor y no he logrado obtener mi certificado de retención a través del portal, ¿a qué correo electrónico debo escribir?
🤖: Puedes comunicarte con el correo **contacto@colombina.com** para resolver cualquier inconveniente con la obtención del certificado de retención a través del portal.
--------------------


--- Pregunta 16/20 ---
❓: ¿Cuál es la política de Colombina respecto al uso de huevos libres de jaula y cuál es la meta para 2025?
🤖: **Política de uso de huevos libres de jaula**  
No se dispone de información en la base de conocimiento sobre una política específica de Colombina respecto al uso de huevos libres de jaula.

**Meta para 2025**  
Igualmente, la base de conocimiento no menciona una meta concreta para el año 2025 en relación con este tema.
--------------------


--- Pregunta 17/20 ---
❓: De acuerdo con la política de tratamiento de datos, ¿qué ocurre con la información de un candidato que no es seleccionado para un puesto de trabajo?
🤖: Según la política de tratamiento de datos de la empresa, la información de un candidato que no es seleccionado para un puesto se conserva de forma archivada durante **30 días**. Ese período sirve para verificar la finalización del proceso y responder a posibles consultas del candidato. Una vez transcurridos esos 30 días, la información es **eliminada o anonimizada**, garantizando que no se mantenga datos personales sin una finalidad justificada.
--------------------


--- Pregunta 18/20 ---
❓: ¿Cuál fue la calificación que recibió Colombina al obtener la certificación Basura Cero ORO para sus plantas de helados?
🤖: La calificación que Colombina obtuvo con la certificación **Basura Cero ORO** para sus plantas de helados fue la propia **“Oro”** (un sello de nivel superior otorgado por ICONTEC y Basura Cero Colombia).
--------------------


--- Pregunta 19/20 ---
❓: Menciona dos deportistas que han sido embajadores o imagen de campañas relacionadas con Bon Bon Bum.
🤖: En la información disponible de Bon Bon Bum se menciona a **Gustavo Puerta** como el embajador oficial de la campaña. No se registra en los documentos una segunda figura deportiva que haya servido como embajador o imagen de la campaña.
--------------------


--- Pregunta 20/20 ---
❓: ¿Cuál es el salario anual del presidente de Colombina?
🤖: No dispongo de esa información.
--------------------

#,Pregunta,Respuesta del LLM,Evaluación,Comentarios

1),¿En qué año se creó el Bon Bon Bum?,"""No dispongo de información...""",🔴 Incorrecta,La información SÍ está en el contexto. El LLM no la encontró.
2),¿Programa para proveedores y emprendedores?,"""Programa Big Brother.""",✅ Correcta,Respuesta perfecta y concisa.
3),¿Porcentaje energía renovable en Colombia?,"""El 100 %...""",✅ Correcta,Extrajo el dato exacto del contexto.
4),¿Certificación de gestión de residuos 5 fábricas?,(Sin respuesta),🔴 Incorrecta,"Falla del modelo. La respuesta (""Basura Cero"") está en el contexto."
5),¿Fundador de Colombina?,"""Carlos Fernando de la Torre.""",🔴 Incorrecta (Alucinación),¡FALLA CRÍTICA! El modelo inventó un nombre. El fundador es Hernando Caicedo.
6),¿Colaboración Bon Bon Bum y Tajín?,(Sin respuesta),🔴 Incorrecta,Falla del modelo. Toda la información estaba en el contexto.
7),¿Logros sostenibilidad energía y agua?,(Tabla detallada con 45% y 27%),✅ Correcta,Excelente respuesta. Encontró y sintetizó los datos correctos.
8),¿Procedimiento actualizar datos personales?,(Lista genérica de 4 pasos),🔴 Incorrecta (Alucinación),,¡FALLA CRÍTICA! El modelo inventó un procedimiento genérico e ignoró el real del PDF (enviar correo a pdp@colombina.com o ir a la Carrera 1 # 24 – 56).
9),¿Historia creación Bon Bon Bum?,"""Lo siento, pero la información... no está disponible.""",🔴 Incorrecta,"Falla del modelo, igual que en la P1. La historia, el creador (Jaime H. Caicedo) y la innovación (relleno de chicle) están en el contexto."
10),¿Qué es Colombina Energía S.A.S. E.S.P.?,(Sin respuesta),🔴 Incorrecta,Falla del modelo. La definición estaba clara en el contexto.
11),¿Comparar alianzas Ramo y Postobón?,"""No se encuentran referencias...""",🔴 Incorrecta,Falla del modelo. Ambas alianzas (Helado de Chocoramo y Bon Bon Bum de Manzana Postobón ) estaban en el contexto.
12),¿Relación Sello Oro Equipares y valores?,(Sin respuesta),🔴 Incorrecta,"Falla del modelo. El contexto vincula explícitamente ""Equipares"" con el valor corporativo de ""Respeto""."
13),"¿Tres plantas, ubicación, productos?","(Tabla con 3 plantas, todas en ""La Paila"")",🔴 Incorrecta (Alucinación),"¡FALLA CRÍTICA! El modelo inventó la información. Las plantas son (1) Confitería en La Paila (dulces, chicles), (2) Galletas en Santander de Quilichao (galletas, waffers), (3) Salsas en Tuluá (salsas, compotas)."
14),¿Plantas fuera de Colombia?,"""...no dispone de plantas de producción fuera de Colombia.""",🔴 Incorrecta,Falla del modelo. El contexto menciona explícitamente plantas en Guatemala y España.
15),¿Email proveedor certificado retención?,"""...contacto@colombina.com""",🔴 Incorrecta (Alucinación),¡FALLA CRÍTICA! El modelo inventó un correo genérico. El correo correcto del PDF es jruano@colombina.com.
16),¿Política huevos libres de jaula?,"""No se dispone de información...""",🔴 Incorrecta,Falla del modelo. El contexto menciona la meta para 2025 de que el 60% de los huevos sean libres de jaula.
17),¿Info candidato no seleccionado?,"""...se conserva... durante 30 días... es eliminada o anonimizada.""",🔴 Incorrecta (Alucinación),"¡FALLA CRÍTICA! El modelo inventó ""30 días"" y ""anonimizada"". El PDF solo dice que ""será eliminada""."
18),¿Calificación Basura Cero ORO helados?,"""...la propia 'Oro'...""",🟡 Parcial,"El modelo no dio la ""calificación"" (el puntaje), sino la categoría. La calificación exacta, 97 sobre 100, sí estaba en el contexto."
19),¿Dos deportistas Bon Bon Bum?,"""...menciona a Gustavo Puerta... No se registra... una segunda figura...""",🟡 Parcial,"El modelo encontró a uno (Gustavo Puerta ) pero falló en encontrar a los otros, como Linda Caicedo o David Alonso y Sebastián Montoya."
20),¿Salario presidente?,"""No dispongo de esa información.""",✅ Correcta (Negativa),¡ÉXITO! Esta era una pregunta trampa. El modelo siguió el prompt y no alucinó.

# Resumen

In [ ]:
import json
from langchain_community.llms import Ollama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

def load_knowledge_base(filepath="chunks.json"):
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)
    context = "\n".join([item["content"] for item in data])
    print(f"✅ Base de conocimiento cargada. {len(data)} fragmentos.")
    return context

def create_summary_chain(knowledge_context, llm_model="gpt-oss:20b"):
    llm = Ollama(model=llm_model)

    template = """
    Eres un asistente experto en redacción y análisis empresarial.
    Genera un resumen claro, preciso y estructurado de la siguiente base de conocimiento de Colombina.
    No inventes información, y divide el resumen en secciones temáticas relevantes.

    --- INICIO DE LA BASE DE CONOCIMIENTO ---
    {context}
    --- FIN DE LA BASE DE CONOCIMIENTO ---

    Resumen:
    """

    prompt = PromptTemplate.from_template(template)

    chain = prompt | llm | StrOutputParser()
    return chain

def main():
    context = load_knowledge_base()
    summary_chain = create_summary_chain(context, llm_model="gpt-oss:20b")

    print("\n--- 🚀 GENERANDO RESUMEN GLOBAL ---\n")
    summary = summary_chain.invoke({"context": context})
    print(summary)

if __name__ == "__main__":
    main()


Resultados

1) --- 🚀 GENERANDO RESUMEN GLOBAL ---

| Tema | Resumen | |------|---------| | **Historia & expansión** | 1976-2024: crecimiento de 50 plantas en 19 países; nuevos negocios en confitería, chocolate, bebidas, comidas rápidas, snacks, jugos, yogures y productos de granja. | | **Sostenibilidad** | • 10 años en *Top 10* de S&P Global por prácticas sostenibles. <br>• Huella de carbono reducida 45 % (2015‑2024). <br>• Agua reducida 27 % (2015‑2024). <br>• 89 % de plantas con certificación Basura Cero. <br>• 56 % de productos libres de colorantes, saborizantes y conservantes artificiales. | | **Innovación & nutrición** | • 2.5 % de ventas en productos propios, 100 % de éstos promueven salud y nutrición. <br>• 64 000 tenderos en Rall‑e Ventas. <br>• Programa Clean & Clear en evolución constante. | | **Economía & desempeño** | • Ventas: $3.3 billones <br>• EBITDA: 14 % <br>• 89 % de colaboradores con contrato directo, 38 % mujeres, 43 % gerenciales. | | **Responsabilidad social** | • Más de 36 000 paneles solares. <br>• Programa Big Brother: 150+ proveedores apoyados. <br>• Fundación Colombina: 13 000 niños, 22 000 participaciones en la Copa Bon Bon Bum 2024. <br>• Sello Oro Equipares y Sello Oro Equipares. | | **Proyectos destacados** | • Granja Solar en La Paila: 100 % de energía renovable, 45 % menos huella de carbono. <br>• Copas deportivas, becas, talleres artísticos y desarrollo comunitario. <br>• Rall‑e Ventas: canal digital para tenderos. | **En una frase**: Colombina combina innovación en alimentos, eficiencia energética y social con un fuerte compromiso con la sostenibilidad, la diversidad y la comunidad.

2) --- 🚀 GENERANDO RESUMEN GLOBAL ---

En la Colombina, todas las plantas que producen productos de alto riesgo (azúcar, maíz, maíz tostado, azúcar de caña, café en polvo con cafeína y chocolate con leche) están sometidas a estrictas normas de seguridad alimentaria y protección ambiental, garantizando el cumplimiento del Código País y las recomendaciones de la Organización de la Salud y la Alimentación (OSHA) y HACCP, lo que se traduce en un 100 % de certificación de seguridad alimentaria en las instalaciones; además, las cinco fábricas de Colombia están certificadas con Basura Cero, lo que implica una estrategia integral de reducción, reutilización y aprovechamiento máximo de residuos orgánicos y sólidos, y han reducido la huella de carbono en un 45 % (por tonelada producida) y el consumo de agua en un 27 %, todo con energía eléctrica 100 % renovable, cumpliendo así con los requisitos de protección ambiental, protección de la salud y nutrición de los consumidores y los colaboradores, y apoyando a la cadena de suministro mediante el programa Big Brother y la certificación de proveedores.

3) --- 🚀 GENERANDO RESUMEN GLOBAL ---

**Resumen Corporativo Responsable – Industria de Alimentos**  

| Año / Etapa | Evento Clave |
|-------------|--------------|
| 1998‑2018 | Adquisición del Grupo 1A (La Paila) |
| 2005 | Compra de la marca Bon Bon |
| 2007 | Compra de la marca La Paila |
| 2008 | Adquisición de la marca Pajarito |
| 2018 | Compra de la marca Colombiana |
| 2019 | Lanzamiento de la marca Bon Bon |
| 2020 | Adquisición de la marca La Paila (plantas) |
| 2022 | Adquisición de la marca Pajarito (plantas) |
| 2024 | Expansión a Chile |

---

**Estrategia de Sostenibilidad (Ejes Temáticos)**  

| Tema | Acción / Programa | Objetivo / Enfoque |
|------|-------------------|--------------------|
| Energía Renovable | Parque solar con 36 000 paneles | Generar el 100 % de la energía consumida a partir de fuentes renovables |
| Gestión de Residuos | Programa “Basura Cero” | Reducir la huella de carbono y reutilizar el 100 % de los excedentes de producción |
| Eficiencia Hídrica | Uso de agua optimizado | Reducir el consumo hídrico en un 27 % respecto a 2015 |
| Salud y Nutrición | Línea “Clean & Clear” | Incrementar la proporción de productos libres de colorantes, saborizantes y conservantes |
| Inclusión y Diversidad | Políticas de igualdad | 38,7 % de la plantilla femenina, 43,6 % de cargos directivos femeninos |
| Gobierno Corporativo | Publicación de informes anuales | Transparencia sobre desempeño financiero y de sostenibilidad |
| Responsabilidad Social | Fundación Colombina | Inversión de 7 300 millones de pesos en iniciativas comunitarias |
| Innovación y Competitividad | Programa “Big Brother” | Capacitación y evaluación de 28 proveedores en criterios de sostenibilidad |

---

**Resultados Clave (Indicadores Medibles)**  

| Área | Métrica | Resultado |
|------|---------|-----------|
| **Balance Ecológico** | Huella de carbono (por tonelada producida) | Reducción de 45 % desde 2015 |
|  | Consumo de energía | Disminución significativa (valor relativo a 2015) |
|  | Consumo de agua | Reducción de 27 % respecto a 2015 |
|  | Uso de residuos | 100 % de plantas con certificación de seguridad alimentaria |
| **Desarrollo Social** | Inversión en programas sociales | 7 300 millones de pesos |
|  | Participación en eventos comunitarios | 22 000 niños en Copa Bon Bon Bum 2024 |
|  | Beneficiados por la Fundación | 13 232 niños y jóvenes |
| **Competitividad** | Replicación de modelos sostenibles con proveedores | 28 proveedores capacitados |
|  | Evaluación de proveedores | 100 % de plantas con certificación de seguridad |
| **Empleados** | Contratos directos | 91,8 % de la plantilla |
|  | Clima organizacional | 89,3 % de satisfacción en encuesta |
|  | Capacitación | 6 800 empleados directos formados |
|  | Inversión en bienestar | 3 600 millones de pesos |
| **Transparencia y Nutrición** | Productos libres de aditivos | 56 % de la línea de productos |
|  | Productos mejorados | 1 438 líneas con enfoque Clean & Clear |
| **Eficiencia Económica** | Ventas anuales | 3 300 billones de pesos |
|  | Margen EBITDA | 14,1 % |
|  | Distribución a tenderers | 64 000 clientes que compran mensualmente vía Rall‑e Ventas |

Este resumen sintetiza las contribuciones económicas, ambientales, sociales y de gobernanza de la corporación, evidenciando su compromiso integral como empresa responsable en la industria alimentaria.

Evaluación técnica de los tres resultados

Resultado 1) — Estructura equilibrada y factual
Este es el mejor de los tres.
Resume con precisión todos los ejes temáticos (historia, sostenibilidad, innovación, desempeño, responsabilidad social y proyectos).
Los datos coinciden exactamente con los de la base de conocimiento (huella -45 %, agua -27 %, EBITDA 14 %, 36 000 paneles, etc.).
Está bien jerarquizado por secciones.
📘 Conclusión:
✅ Es el resumen más fiel y equilibrado.
Perfecto para usar como “fuente” para la fase de reformulación fluida.

Resultado 2) — Excesivamente técnico y específico
Se centra casi exclusivamente en seguridad alimentaria y Basura Cero.
Ignora innovación, desempeño financiero y proyectos sociales.
Parece que el modelo se “atascó” en una de las secciones de la base y amplificó ese contexto.
📘 Conclusión:
⚠️ No es representativo globalmente. Descártalo para el resumen final.

Resultado 3) — Resumen tipo informe técnico / ESG
Muy detallado, bien estructurado, tipo “reporte de sostenibilidad”.
Tiene cifras adicionales (7 300 millones, 6 800 empleados, 1 438 productos, etc.) que sí están en la base, pero el estilo tabular es más rígido.
Es útil si quisieras generar un anexo técnico, pero no como texto narrativo.
📘 Conclusión:
✅ Muy completo, pero demasiado extenso y segmentado.
Podría servir para fusionar cifras si quieres una versión “enriquecida”.

# FAQ

In [ ]:
import json
from langchain_community.llms import Ollama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


# ==========================================================
# 1️⃣ Cargar y limitar la base de conocimiento
# ==========================================================
def load_knowledge_base(filepath="chunks.json", max_chunks=25):
    """
    Carga el archivo con los chunks del conocimiento y selecciona los más relevantes
    para evitar exceder el límite de contexto del modelo.
    """
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Selecciona los fragmentos más largos (los más informativos)
    selected = sorted(data, key=lambda x: len(x["content"]), reverse=True)[:max_chunks]
    context = "\n".join([item["content"] for item in selected])

    print(f"✅ Base de conocimiento cargada. {len(selected)} fragmentos seleccionados de {len(data)} totales.")
    print(f"📏 Tamaño aproximado del contexto: {len(context.split())} palabras.\n")
    return context


# ==========================================================
# 2️⃣ Crear la cadena de generación de preguntas FAQ
# ==========================================================
def create_faq_chain(knowledge_context, llm_model="gpt-oss:20b"):
    """
    Crea una cadena LangChain que genera automáticamente preguntas frecuentes (FAQ)
    sobre Colombina, basadas en la base de conocimiento.
    """
    llm = Ollama(model=llm_model)

    template = """
    Eres un generador de Preguntas Frecuentes (FAQ) sobre la empresa **Colombina**, 
    un líder latinoamericano en alimentos. Tu tarea es analizar la base de conocimiento 
    provista y producir directamente entre 15 y 25 preguntas realistas y diversas 
    que un cliente, proveedor o colaborador podría hacer sobre la empresa.

    Reglas:
    - Las preguntas deben centrarse en Colombina: su historia, productos, sostenibilidad, 
      innovación, programas sociales, proveedores y contacto.
    - No generes preguntas genéricas sobre liderazgo o gestión.
    - No pidas aclaraciones al usuario ni solicites más información.
    - No incluyas respuestas ni formato adicional, solo las preguntas enumeradas.
    - Redacta las preguntas en tono formal y natural, como en una sección de FAQ corporativa.

    --- INICIO DE LA BASE DE CONOCIMIENTO ---
    {context}
    --- FIN DE LA BASE DE CONOCIMIENTO ---

    Preguntas frecuentes:
    """

    prompt = PromptTemplate.from_template(template)
    chain = prompt | llm | StrOutputParser()
    return chain


# ==========================================================
# 3️⃣ Función principal
# ==========================================================
def main():
    # Cargar base de conocimiento limitada
    context = load_knowledge_base(filepath="chunks.json", max_chunks=25)

    # Crear la cadena FAQ
    faq_chain = create_faq_chain(context, llm_model="gpt-oss:20b")

    print("\n--- 🚀 GENERANDO PREGUNTAS FRECUENTES DE COLOMBINA ---\n")

    # Invocar el modelo
    faqs = faq_chain.invoke({"context": context})
    print(faqs)

    # Guardar resultado
    output_file = "faqs_colombina.txt"
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(faqs)

    print(f"\n✅ Preguntas frecuentes guardadas en '{output_file}'.")


# ==========================================================
# 4️⃣ Ejecución directa
# ==========================================================
if __name__ == "__main__":
    main()


--- 🚀 GENERANDO PREGUNTAS FRECUENTES DE COLOMBINA ---

¡Claro! Aquí tienes una sección de preguntas frecuentes (FAQ) que cubre los temas más solicitados sobre **Colombina** y su ecosistema de productos, sostenibilidad, operaciones y programas corporativos. Si tienes alguna duda específica que no aparezca aquí, no dudes en decírmelo.

---

## Preguntas Frecuentes (FAQ)

| # | Pregunta | Respuesta |
|---|----------|-----------|
| **1** | ¿Cuáles son las marcas más representativas de Colombina? | Colombina gestiona un portafolio de más de 140 marcas, entre las más conocidas se encuentran: <br>• *Amazon* (salsas y ají picante)<br>• *Colombina 100%* (productos libres de colorantes y saborizantes artificiales)<br>• *Tajín* (sazonador)<br>• *Ramo* (snacks)<br>• *Búfalo*, *Bésame*, *Búfalo*, *Joya*, *Chocolates y más. |
| **2** | ¿Dónde se producen los productos de Colombina? | Colombina opera 5 fábricas certificadas por **Basura Cero** en Colombia y 2 plantas de helados con la certificación **Basura Cero Oro**. También tiene plantillas en República Dominicana, España, y otras filiales en Centroamérica, Europa y África. |
| **3** | ¿Qué iniciativas de sostenibilidad lidera Colombina? | <ul><li>**Basura Cero Oro** (certificación de 97/100).</li><li>Reducción de agua en un 27 % desde 2015.</li><li>Uso de 70 % de azúcar de caña en la producción de 3 tipos de azúcar.</li><li>Plan de gestión de residuos sólidos y orgánicos con un enfoque “cero basura”.</li><li>Participación en el **Top 10** de empresas de alimentos con mejores prácticas sostenibles de S&P Global 2024.</li></ul> |
| **4** | ¿Qué es la Copa Bon Bon Bum? | Es una competencia deportiva para más de 22,000 niños y familias cada año, fomentando el deporte y la recreación. Colombina patrocina la copa y tiene un embajador deportivo (Gustavo Puerta) que inspira a los participantes. |
| **5** | ¿Qué programas de bienestar ofrece a sus colaboradores? | • Programa *Pays* (pensionado, acompañamiento y seguro).<br>• Ajustes de vivienda (1 405 personas han adquirido vivienda propia).<br>• Desarrollo profesional, salud, seguridad y bienestar laboral. |
| **6** | ¿Cuáles son los valores corporativos de Colombina? | • Trabajo en Equipo<br>• Compromiso<br>• Orientación al Cliente<br>• Respeto |
| **7** | ¿Qué tipos de alimentos y snacks produce Ramo? | Ramo produce ponqués, pasabocas y galletas, con enfoque en innovación y tecnología. Colaboradores: 3 000 empleados, presencia nacional con dos plantas en Mosquera, Cundinamarca. |
| **8** | ¿Cómo se involucra Colombina con la comunidad? | Fundación Colombina: Donaciones de 200 000 pesos para programas de apoyo a comunidades necesitadas. Programa *Ramo* celebra 75 años de tradición en 2025. |
| **9** | ¿Qué certificaciones de diversidad y equidad posee la compañía? | Colombina fue primera en recibir el **Sello Oro Equipares** y mantiene la certificación durante 6 años. |

---

**Fortalezas claras**

Está contextualizado correctamente
Todas las preguntas giran en torno a Colombina: productos, sostenibilidad, programas, valores, colaboradores y comunidad.
Ya no aparecen temas genéricos como “liderazgo” o “KPIs”.
Esto demuestra que el prompt ancló correctamente el dominio.

Incluye datos y hechos reales
Cita correctamente indicadores de tu base: reducción del 27 % de agua, certificación Basura Cero Oro, Sello Oro Equipares, Copa Bon Bon Bum, presencia internacional.
Estos puntos existen textualmente en tu corpus — así que está basándose en la fuente correcta.

Estructura de FAQ limpia y funcional
Formato Markdown tabular: usable directamente en tu interfaz Streamlit o en tu informe.

Genera 9 FAQs con contenido detallado; cada una incluye pregunta + respuesta estructurada.
Demuestra comprensión semántica de múltiples ejes
Integra sostenibilidad, expansión, bienestar laboral, comunidad y valores corporativos.
Usa vocabulario corporativo consistente con informes reales de Colombina.

**Ajustes sugeridos**

Filtración leve de marcas externas
Menciona Ramo y Tajín, que sí aparecen en tu base de conocimiento (alianzas y productos compartidos),
pero se mezclan con “Amazon” — ese es un falso positivo generado por ruido contextual.

